## Zadanie - GUS DBW
To ćwiczenie ma pokazać nam jak krok po kroku kodować programy pobierające dane przez API, aby minimalizować ryzyko strat czasu / danych. Podczas ściągania dużo rzeczy może pójść nie tak np. zerwanie połączenia, blokada ze strony serwera etc. Dlatego chcemy zapisywać treść w międzyczasie.

### Polecenie 1:
Będziemy pracowali z danymi [GUS](https://api-dbw.stat.gov.pl/apidocs/index.html). W pierwszej kolejności przygotujmy funkcję, która zgodnie z dokumentacją pobierze listę dostępnych zmiennych w katalogu. Wykorzystujemy metodę ***area/area-area***.

Najpierw uruchom ten blok kodu:

In [ ]:
import pandas as pd
import urllib
URL = "https://api-dbw.stat.gov.pl/api/1.1.0/"

Teraz wygeneruj końcowy URL w bloku poniżej:

In [ ]:
metoda_API = "area/area-area"
final_URL = URL + metoda_API
final_URL

### Polecenie 2:
Otwórz za pomocą pandas dane z linku - przyda się metoda *read_json*. Znajdź id dziedziny w której opisywane są ceny nieruchomości.

In [ ]:
json_df = pd.read_json(final_URL)
json_df

### Polecenie 3:
Wyświetl liczbę zmiennych znajdujących się w tej dziedzinie. Do tego będzie służyć metoda API **area/area-variable**. Najprościej zrobić to zmieniając link o jaki pytaliśmy w ramce.

W tym momencie będziemy przekazywać parametry do zapytania. Przyda nam się słownik oraz metoda *urllib.parse.urlencode*, która zmieni jego zawartość na link do HTML.

In [ ]:
metoda_API = "area/area-variable"

parametry = {
    "lang":"pl",
    "id-obszaru": "12"
}
final_URL = URL + metoda_API + "?" + urllib.parse.urlencode(parametry)
final_URL

In [ ]:
json_df = pd.read_json(final_URL)
json_df

### Polecenie 4:
Przyjrzyjmy się średnim cenom za 1m2 lokali.  
W normalnym API powinno być tak, że pokazujemy id-zmiennej. GUS to jednak specyficzna instytucja - faktycznie trzeba przejść lekką gimnastykę. Zaczynamy od tego, aby sprawdzić metadane - metoda **variable/variable-meta**

In [ ]:
metoda_API = "variable/variable-meta"
parametry = {
    "lang":"pl",
    "id-zmiennej": "309",
}
final_URL = URL + metoda_API + "?" + urllib.parse.urlencode(parametry)
final_URL

Pobranie danych przez pandas nie wyjdzie - dane nie mają charakteru ramki. To duży błąd twórców. Niemniej możliwy jest lekki workaround przy pomocy bibliotek requests i json:

In [ ]:
import requests
import json

dane = requests.get(final_URL)
json_data = json.loads(dane.text)
json_data

Normalnie potrzeba też znaleźć id okreśów - trzeba korzystać z metody: **dictionaries/periods-dictionary**.

Wykoryzstamy już pobrane informacje:
*   Danym za I kwartał odpowiada liczba 270
*   Za II kwartał - 271
*   Za III kwartał - 272
*   Za IV kwartał - 273

Wykonajmy prośbę o danych za I kwartał 2023 r. Wykorzystujemy metodę: **variable/variable-data-section**, *id-przekroju* probieramy z metadanych

In [ ]:
metoda_API = "variable/variable-data-section"
parametry = {
    "lang":"pl",
    "id-zmienna": "309",
    "id-przekroj": "485",
    "id-rok": "2023",
    "id-okres": "270",
    "numer-strony": "0",
    "ile-na-stronie":50
}
final_URL = URL + metoda_API + "?" + urllib.parse.urlencode(parametry)
final_URL

In [ ]:
dane = requests.get(final_URL)
json_data = json.loads(dane.text)
json_data

Przepisz dane do ramki *pandas*:

In [60]:
final_df = pd.DataFrame(json_data['data'])
final_df

,rownumber,id-zmienna,id-przekroj,id-wymiar-1,id-pozycja-1,id-wymiar-2,id-pozycja-2,id-okres,id-sposob-prezentacji-miara,id-daty,id-brak-wartosci,id-tajnosci,id-flaga,wartosc,precyzja
0,1,309,485,10,33617,423,4801795,270,180,2021,253,43,36,6792.0,2
1,2,309,485,10,33617,423,4801796,270,180,2021,253,43,36,5719.0,2
2,3,309,485,10,33617,423,4801797,270,180,2021,253,43,36,6214.0,2
3,4,309,485,10,33619,423,4801795,270,180,2021,253,43,36,7734.0,2
4,5,309,485,10,33619,423,4801796,270,180,2021,253,43,36,7496.0,2
5,6,309,485,10,33619,423,4801797,270,180,2021,253,43,36,7630.0,2
6,7,309,485,10,33622,423,4801795,270,180,2021,253,43,36,5146.0,2
7,8,309,485,10,33622,423,4801796,270,180,2021,253,43,36,4759.0,2
8,9,309,485,10,33622,423,4801797,270,180,2021,253,43,36,4928.0,2
9,10,309,485,10,33634,423,4801795,270,180,2021,253,43,36,5558.0,2


### Polecenie 5:
Wykonaj w pętli pytania dla lat 2021-2022 i dla wszystkich kwartałów - każdorazowo łącz dane oraz zapisuj je w odrębnych Excelach

In [57]:
import time

In [62]:
lata = [2021, 2022]
id_okresow = [270, 271, 272, 273]
metoda_API = "variable/variable-data-section"
parametry = {
    "lang":"pl",
    "id-zmienna": "309",
    "id-przekroj": "485",
    "id-rok": "2023",
    "id-okres": "270",
    "numer-strony": "0",
    "ile-na-stronie":50
}

for i in lata:
  parametry["id-rok"] = i
  for j in id_okresow:
    parametry["id-okres"] = j
    final_URL = URL + metoda_API + "?" + urllib.parse.urlencode(parametry)
    dane = requests.get(final_URL)
    json_data = json.loads(dane.text)
    json_df = pd.DataFrame(json_data['data'])
    json_df.to_excel("CenyM2_" + str(i) + "_" + str(j) +".xlsx")

    final_df = pd.concat([final_df, json_df])
    time.sleep(5)

In [63]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 49
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   rownumber                    450 non-null    int64  
 1   id-zmienna                   450 non-null    int64  
 2   id-przekroj                  450 non-null    int64  
 3   id-wymiar-1                  450 non-null    int64  
 4   id-pozycja-1                 450 non-null    int64  
 5   id-wymiar-2                  450 non-null    int64  
 6   id-pozycja-2                 450 non-null    int64  
 7   id-okres                     450 non-null    int64  
 8   id-sposob-prezentacji-miara  450 non-null    int64  
 9   id-daty                      450 non-null    int64  
 10  id-brak-wartosci             450 non-null    int64  
 11  id-tajnosci                  450 non-null    int64  
 12  id-flaga                     450 non-null    int64  
 13  wartosc              